In [62]:
import pandas as pd
import numpy as np
from datetime import datetime
import utils
import importlib

importlib.reload(utils)

<module 'utils' from '/Users/devyn/Computer Science/Data Science/da8_personal_project/utils.py'>

# Personal Strava EDA  

Write introduction and author here...

Write about `data` and attributes here...

## Cleaning the Data  

Write the ways in which you are cleaning the data...  

In [63]:
utils.collectWeatherData()
utils.collectStravaData()

/Users/devyn/Computer Science/Data Science/da8_personal_project/utils.py:133: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
